In [1]:
import json
import linecache
import time
import jsonlines
from datetime import datetime, date
import pickle

## read biorxiv_json

In [2]:
all_paper_full_infos=[] ### store all papers from bioxiv, chem, med, arxiv 

biorxiv_json = 'biorxiv-metadata-oai-snapshot.json'
starting_date = date(1990,1,1)
start_time = time.time()

with jsonlines.open(biorxiv_json, 'r') as f:
    for id_of_abstract, line in enumerate(f):
        get_date = datetime.strptime(line['date'], '%Y-%m-%d').date()
        paper_time = (get_date - starting_date).days
        all_paper_full_infos.append([line['server'],line['title'],line['abstract'],paper_time])

num1=len(all_paper_full_infos)
elapsed_time = time.time() - start_time
print(f"Current: {len(all_paper_full_infos)}; Read biorxiv: {len(all_paper_full_infos)}, Elapsed time: {elapsed_time} seconds")

Current: 184839; Read biorxiv: 184839, Elapsed time: 2.6651909351348877 seconds


## read medrxiv_json

In [3]:

medrxiv_json = 'medrxiv-metadata-oai-snapshot.json'

start_time = time.time()
with jsonlines.open(medrxiv_json, 'r') as f:
    for id_of_abstract, line in enumerate(f):
        get_date = datetime.strptime(line['date'], '%Y-%m-%d').date()
        paper_time = (get_date - starting_date).days
        all_paper_full_infos.append([line['server'],line['title'],line['abstract'],paper_time])

num2=len(all_paper_full_infos)
elapsed_time = time.time() - start_time
print(f"Current: {len(all_paper_full_infos)}; Read medrxiv: {len(all_paper_full_infos)-num1}, Elapsed time: {elapsed_time} seconds")

Current: 224071; Read medrxiv: 39232, Elapsed time: 0.6739270687103271 seconds


## read chemrxiv_json

In [4]:

chemrxiv_json = 'chemrxiv-metadata-oai-snapshot.json'

start_time = time.time()
with jsonlines.open(chemrxiv_json, 'r') as f:
    for id_of_abstract, line in enumerate(f):
        get_date = datetime.strptime(line['date'][:10], '%Y-%m-%d').date()
        paper_time = (get_date - starting_date).days
        all_paper_full_infos.append([line['server'],line['title'],line['abstract'],paper_time])

num3=len(all_paper_full_infos)
elapsed_time = time.time() - start_time
print(f"Current: {len(all_paper_full_infos)}; Read chemrxiv: {len(all_paper_full_infos)-num2}, Elapsed time: {elapsed_time} seconds")

Current: 240551; Read chemrxiv: 16480, Elapsed time: 0.25910282135009766 seconds


## check duplicates papers 

In [47]:
### check whether there are some duplicates papers might due to api re-requesting; 
duplicates = {}
for i, paper_info in enumerate(all_paper_full_infos):
    # Convert the sublist to a hashable type (e.g. tuple) to use as a key
    key = tuple(sorted(map(str, paper_info)))
    if key in duplicates:
        duplicates[key].append(i)
    else:
        duplicates[key] = [i]

# Print the duplicates
count=0
for key, indices in duplicates.items():
    if len(indices) > 1:
        #print(f"{len(indices)} instances of {list(key)}")
        #print(f"Indices: {indices}")
        count+=1
        
print(f"remove duplicates: {count}") ## for len(indices)>2, need to remove 2; so +1

remove duplicates: 27


In [23]:
## remove repeated papers (there are some repeated papers)
paper_infos_unique = set(map(tuple, all_paper_full_infos)) # convert each sublist to a tuple and create a set
all_paper_infos_unique = list(map(list, paper_infos_unique)) # convert each tuple back to a list and create a list

print(f"remove duplicates: {len(all_paper_full_infos)-len(all_paper_infos_unique)}")

with open('all_paper_info_lists_bio_med_chem.pkl', 'wb') as f:
    pickle.dump(all_paper_infos_unique, f)

remove duplicates: 28


## read arxiv_json 

In [24]:
start_time = time.time()
arxiv_json = 'arxiv-metadata-oai-snapshot.json'

with jsonlines.open(arxiv_json, 'r') as f:
    for id_of_abstract, line in enumerate(f):
        get_date = datetime.strptime(line['versions'][0]['created'], '%a, %d %b %Y %H:%M:%S %Z').date()
        paper_time = (get_date - starting_date).days
        all_paper_infos_unique.append(['arxiv',line['title'],line['abstract'],paper_time])
        
elapsed_time = time.time() - start_time
print("arxiv: {id_of_abstract}")
print(f"Current: {len(all_paper_infos_unique)}; Read chemrxiv: {len(all_paper_infos_unique)-num3}, Elapsed time: {elapsed_time} seconds")

arxiv: {id_of_abstract}
Current: 2444442; Read chemrxiv: 2203891, Elapsed time: 44.237696170806885 seconds


In [ ]:
import pickle
with open('all_paper_info_lists.pkl', 'wb') as f:
    pickle.dump(all_paper_infos_unique, f)

## some test

In [ ]:
'''
with open('all_paper_info_lists.pkl', 'rb') as f:
    all_paper_infos_test=pickle.load(f)
    
x_set = set(map(tuple, all_paper_infos_test))
y_set = set(map(tuple, all_paper_infos_test2))

if x_set == y_set:
    print("The lists have the same sublists")
else:
    print("The lists have different sublists")
'''